In [ ]:
import os
import requests
import re
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, tool, Tool

# Weather Tool
OPENWEATHER_API_KEY = "bd5e378503939ddaee76f12ad7a97608"
SPOONACULAR_API_KEY = "f2f6b1736b0e488d9bf536af6b017519"
BASE_URL = "https://api.spoonacular.com"

categories = ['Weather', 'Fitness', 'Wellbeing', 'Sleep', 'Calendar', 'Nutrition', 'Reminder', 'Search', 'Unknown']

# Categories and descriptions
category_descriptions = {
    "Weather": "query include weather-related intent",
    "Fitness": "query asks about exercise or fitness routines",
    "Wellbeing": "query shares emotional, mood, or journal-related inputs but not general questions",
    "Sleep": "query talks about sleep, fatigue, or rest",
    "Calendar": "query refers to appointments or events",
    "Nutrition": "query include meal names",
    "Reminder": "query include intent with reminding something",
    "Search": "query include a question but it is not related to the other categories",#  as long as it is not related to weather, fitness, wellbeing, sleep, calender, nutrition, nor reminder",
}

# Build the classification system prompt
desc_lines = "\n".join(f"{cat}: {desc}" for cat, desc in category_descriptions.items())
prompt_router = f"""
You are a classification assistant. Your ONLY task is to analyze user's paragraph then split it into parts then categorize them based on the categories provided using their description.
CATEGORIES:
{desc_lines}

Instructions:
1) Read the full paragraph, analyze it and split it all into parts.
2) For each part, decide which category (or multiple) it belongs to based on intent and your understanding from the predefined CATEGORIES making sure they fit the descriptions provided.
3) If a part does not fit any of the predefined CATEGORIES, then put it as UNKNOWN category.
4) DO NOT provide answers, suggestions, extra commentary, or explanations.
5) DO NOT return anything except classified part under each category name as the OUTPUT FORMAT.
6) Make sure all the input text was split properly and classified, so nothing is missing from the input

OUTPUT FORMAT:
category_name:
- [Sentence].
"""

@tool
def get_weather(data: dict, ctx: dict) -> dict:
    """
    Fetch current weather from OpenWeatherMap and update context.

    Args:
        data (dict): Contains 'location' with 'lat' and 'lon'.
        ctx (dict): User context to update.

    Returns:
        dict: Weather info including 'city', 'temp', 'condition', and 'wind_kph'.
    """
    loc = data.get('location', {})
    lat = loc.get('lat')
    lon = loc.get('lon')
    if lat is None or lon is None:
        return {"error": "Location not provided"}
    key = os.getenv("OPENWEATHER_API_KEY", OPENWEATHER_API_KEY)
    try:
        r = requests.get(
            "https://api.openweathermap.org/data/2.5/weather",
            params={"lat": lat, "lon": lon, "units": "metric", "appid": key},
            timeout=5
        ).json()
        info = {
            "city": r.get("name", ""),
            "temp": r["main"]["temp"],
            "condition": r["weather"][0]["description"],
            "wind_kph": round(r["wind"]["speed"] * 3.6, 1)
        }
    except Exception as e:
        return {"error": str(e)}
    ctx['weather'] = info
    return info
class SpoonacularTool(Tool):
    inputs = {
        "input_text": {
            "type": "string",
            "description": (
                "Command: 'search:<query>' or 'details:<id>'"
            )
        }
    }
    output_type = "string"
    name = "SpoonacularTool"
    description = (
        "Use this tool to search recipes or fetch full details: instructions, nutrition, taste"
    )

    def forward(self, input_text: str) -> str:
        command, _, arg = input_text.partition(":")
        if command == "search":
            return self.search_recipes(arg)
        if command == "details":
            return self.get_full_details(arg)
        return f"Unknown command: {input_text}"

    def search_recipes(self, query: str, number: int = 3) -> str:
        url = f"{BASE_URL}/recipes/complexSearch"
        params = {
            'query': query,
            'number': number,
            'addRecipeInformation': False,
            'apiKey': SPOONACULAR_API_KEY
        }
        resp = requests.get(url, params=params)
        resp.raise_for_status()
        results = resp.json().get('results', [])
        return "\n".join(f"{r['id']}: {r['title']}" for r in results)

    def get_full_details(self, recipe_id: str) -> str:
        # Fetch instructions
        instr = requests.get(
            f"{BASE_URL}/recipes/{recipe_id}/analyzedInstructions",
            params={'apiKey': SPOONACULAR_API_KEY}
        ).json()
        steps = [step['step'] for section in instr for step in section.get('steps', [])]
        # Fetch nutrition
        nut = requests.get(
            f"{BASE_URL}/recipes/{recipe_id}/nutritionWidget.json",
            params={'apiKey': SPOONACULAR_API_KEY}
        ).json()
        # Fetch taste
        taste = requests.get(
            f"{BASE_URL}/recipes/{recipe_id}/tasteWidget.json",
            params={'apiKey': SPOONACULAR_API_KEY}
        ).json()
        # Assemble output
        out = "**Instructions:**\n"
        for i, s in enumerate(steps, 1): out += f"{i}. {s}\n"
        out += "\n**Nutrition per serving:**\n"
        out += f"- Calories: {nut['calories']}\n"
        out += f"- Carbs: {nut['carbs']}\n"
        out += f"- Fat: {nut['fat']}\n"
        out += f"- Protein: {nut['protein']}\n\n"
        out += "**Taste Profile:**\n"
        for k, v in taste.items(): out += f"- {k.capitalize()}: {v}%\n"
        return out
# Prompt generator
model = LiteLLMModel(model_id="ollama/qwen2.5-coder:3b")

def generate_prompt(name, purpose, capabilities, returns, final_only=True):
    parts = [
        f"You are {name}.",
        f"Purpose: {purpose}",
        "Capabilities:",
        *[f"- {c.strip()}" for c in capabilities.split(';') if c.strip()],
        "Returns:",
        *[f"- {r.strip()}" for r in returns.split(';') if r.strip()],
        "Only output the final result in plain text."
    ]
    if final_only:
        parts.append("Do not show any code, plan, or explanation.")
    parts.append("If you cannot respond, return 'NO_RESPONSE'.")
    return "\n".join(parts)

# Define prompts
prompt_wellbeing = generate_prompt(
    "Wellbeing Companion",
    "Provide psychological support and mental wellness tips based on user journal or mood inputs.",
    "Analyze emotional tone; Suggest wellness activities; Support stress and anxiety",
    "Short-term coping tips; Actionable wellness advice"
)

prompt_fitness = generate_prompt(
    "Fitness Coach",
    "Provides tailored workout recommendations based on user requests and environmental factors to have a complete workout or help the user with his fitness requests.",
    "Use weather and activity history; Recommend suitable fitness activities and help the user with his requests",
    "A personalized activity suggestion"
)

prompt_search = generate_prompt(
    "Search Assistant",
    "Search for answers to user queries when others cannot respond.",
    "Perform web search using DuckDuckGoSearchTool; Summarize key result",
    "Concise factual answer"
)

prompt_nutrition = (
    "You are an expert cooking assistant. When given any user query about meals, you must: \n"
    "1) Read the query and extract all dish names or food items (e.g., 'pizza'). \n"
    "2) For each extracted dish, use the tool with 'search:<dish>' to retrieve top recipe IDs and titles. \n"
    "3) Choose the most relevant recipe ID for each dish. \n"
    "4) For each dish, use the tool with 'details:<id>' to fetch full instructions, nutrition, and taste profile. \n"
    "5) Present a combined response organized by dish, including recipe name, steps, nutrition, and flavor profile. \n"
    "6) If the you were not able to get results from the SpoonacularTool use DuckDuckGoSearchTool to provide the answer."
)


# Initialize agents
agent_wellbeing = CodeAgent(model=model, tools=[], max_steps=2, verbosity_level=-1)
agent_fitness = CodeAgent(model=model, tools=[get_weather], max_steps=2, verbosity_level=-1)
agent_search = CodeAgent(model=model, tools=[DuckDuckGoSearchTool()], max_steps=2, verbosity_level=-1)
agent_nutrition = CodeAgent(model=model, tools=[SpoonacularTool(), DuckDuckGoSearchTool()], max_steps=2, verbosity_level=-1)
agent_nutrition.prompt_templates['system_prompt'] = prompt_nutrition

agent_router = CodeAgent(model=model, tools=[], max_steps=3, verbosity_level=-1)

# Output dictionary setup
agent_prompt = {cat: [] for cat in categories}
agent_prompt["Wellbeing"] = [agent_wellbeing, prompt_wellbeing]
agent_prompt["Fitness"] = [agent_fitness, prompt_fitness]
agent_prompt["Search"] = [agent_search, prompt_search]
agent_prompt["Nutrition"] = [agent_nutrition, prompt_nutrition]


def router_helper(raw_outputs):
    print("RAW OUTPUTS: \n", raw_outputs)
    # Categories list
    categories = ['Weather', 'Fitness', 'Wellbeing', 'Sleep', 'Calendar', 'Nutrition', 'Reminder', 'Search', 'Unknown']

    # Output dictionary setup
    output_dict = {cat: [] for cat in categories}
    # Extract each category block


    # Extract each category block
    blocks = re.split(r'\n(?=[A-Za-z]+:)', raw_outputs.strip())

    for block in blocks:
        match = re.match(r'([A-Za-z]+):\s*(.*)', block, re.DOTALL)
        if not match:
            continue
        category, content = match.groups()
        category = category.strip().capitalize()

        if category not in output_dict:
            continue

        # Extract both bracketed sentences and those starting with "-"
        bracket_sentences = re.findall(r'\[([^\[\]]+?)\]', content)
        dash_sentences = re.findall(r'-\s*([^\n]+)', content)

        # Combine and clean
        all_sentences = bracket_sentences + dash_sentences

        for sentence in all_sentences:
            sentence = sentence.replace(']',"").replace('[',"").strip().rstrip('.').lower()
            if sentence not in output_dict[category]:
                output_dict[category].append(sentence)


    combined_output = {k: ', & '.join(v) for k, v in output_dict.items()}
    print("combined_output OUTPUTS: ", combined_output)
    return combined_output

def agent_invoker(input_text, weather_text, prompt_type, agent_type, curr_agent):
    enriched = f"{prompt_type}\nWeather: {weather_text}\nUser: {input_text}"
    print(f"\n[Invoking {agent_type} Agent]")
    response = curr_agent.run(enriched).strip()
    if response != 'NO_RESPONSE':
        pass
    print(f"\n[{agent_type} Agent Response]", response)
    return response

# Router
def router(user_input: str, location: dict) -> str:
    ctx = {}

    print("\n--- ROUTER INPUT ---")
    print(f"Input: {user_input}")

    # Weather
    weather_info = get_weather({"location": location}, ctx)
    if "error" not in weather_info:
        weather_text = (
            f"Current weather in {weather_info['city']}: {weather_info['temp']}°C, "
            f"{weather_info['condition']}"
        )
    else:
        weather_text = "Weather info unavailable"
    print("\n[Weather Context]", weather_text)

    full_prompt = prompt_router + "\nUser Input: " + user_input
    
    # Segment and classify
    combined_output = router_helper(agent_router.run(full_prompt))
    all_responses = []
    for k, v in combined_output.items():
        # Compose and run wellbeing agent
        if v:
            if k in ["Wellbeing", "Fitness", "Search"]:
                response = agent_invoker(v, weather_text, agent_prompt[k][1], k, agent_prompt[k][0])
            elif k == ["Nutrition"]:
                print(f"\n[Invoking Nutrition Agent]")
                response = agent_nutrition.run(v)
                print(f"\n[Nutrition Agent Response]", response)
            else: # Default to search
                response = agent_invoker(v, weather_text, prompt_search, "Search", agent_search)
            
            all_responses.append(response)
            

# --- Test ---
if __name__ == '__main__':
    test_input = "I am feeling energetic today, I want to do a good workout that includes going" \
    " to the swimming pool due to it being very hot today," \
    " I want to know what does the word key mean in German, also add to my training biking and" \
    " tell me a good place to do it. I need to eat to boost my protein intake"
    location = {'lat': 48.8566, 'lon': 2.3522}  # Paris
    output = router(test_input, location)
    # print(output)